In [ ]:
import csv
import os

import pandas as pd
import torch
from PIL import Image
from tqdm.notebook import tqdm
from transformers import CLIPModel, CLIPProcessor, pipeline

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define class labels
class_labels = [
    "Hotel Exterior or outdoor building",
    "Hotel Room or living room or bedroom",
    "Pool or swimming pool",
    "Billiard or pool table",
    "Bathroom or toilet",
    "Restaurant or the dining room at the hotel",
    "Hotel Lobby or reception area,",
    "Beach or shore or coastline",
    "Corridors and stairs",
    "Food or food on a plate or food on the table",
    "Conference room or a meeting room",
    "Gym or fitness center or exercise machine",
    "Balcony",
    "Terrace and patio",
    "Spa and sauna or massage center or turkish bath",
]

# Threshold for the "Остальное" category
similarity_threshold = 0.5  # Adjust this value as needed

# Folder of images to classify
image_folder = "public_test"


# Create a function to classify images
def classify_image(image_path):
    image = Image.open(image_path)

    # Preprocess the image and text (class labels)
    inputs = processor(text=class_labels, images=image, return_tensors="pt", padding=True)

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract logits and convert to probabilities
    logits_per_image = outputs.logits_per_image  # Shape: (1, num_labels)
    probs = logits_per_image.softmax(dim=1)  # Convert logits to probabilities

    # Get the predicted class and its probability
    max_prob, predicted_idx = probs.max(dim=1)
    predicted_class = class_labels[predicted_idx.item()]

    # Check if the probability is below the threshold
    if max_prob.item() < similarity_threshold:
        return "Остальное", 16  # Classify as "Остальное" with class number 16
    else:
        return (
            predicted_class,
            predicted_idx.item() + 1,
        )  # Return class number (1-based index)


# List to store results for CSV
csv_results = []

# Loop over images in the folder and classify them
for filename in tqdm(os.listdir(image_folder)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        predicted_class, class_number = classify_image(image_path)
        csv_results.append([filename, class_number])
        print(
            f"Image: {filename}, Predicted Class: {predicted_class}, Class Number: {class_number}"
        )

csv_results.sort(key=lambda x: int(x[0].split("_")[2].split(".")[0]))
# Save the results to a CSV file
csv_file = "classification_results_large_openai.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["image_name", "class_number"])  # Write header
    writer.writerows(csv_results)  # Write image classifications

  0%|          | 0/1124 [00:00<?, ?it/s]

Image: public_test_432.jpg, Predicted Class: Hotel Room or living room or bedroom, Class Number: 2
Image: public_test_121.jpg, Predicted Class: Balcony, Class Number: 13
Image: public_test_930.jpg, Predicted Class: Hotel Lobby or reception area,, Class Number: 7
Image: public_test_1118.jpg, Predicted Class: Hotel Lobby or reception area,, Class Number: 7
Image: public_test_593.jpg, Predicted Class: Terrace and patio, Class Number: 14
Image: public_test_636.jpg, Predicted Class: Остальное, Class Number: 16
Image: public_test_27.jpg, Predicted Class: Hotel Lobby or reception area,, Class Number: 7
Image: public_test_730.jpg, Predicted Class: Bathroom or toilet, Class Number: 5
Image: public_test_265.jpg, Predicted Class: Corridors and stairs, Class Number: 9
Image: public_test_581.jpg, Predicted Class: Hotel Room or living room or bedroom, Class Number: 2
Image: public_test_410.jpg, Predicted Class: Restaurant or the dining room at the hotel, Class Number: 6
Image: public_test_278.jpg, P

KeyboardInterrupt: 

In [ ]:
# load pipe
image_classifier = pipeline(
    task="zero-shot-image-classification", model="openai/clip-vit-large-patch14"
)

image = Image.open("public_test/public_test_700.jpg")

# inference
outputs = image_classifier(image, candidate_labels=class_labels)
outputs = [
    {"score": round(output["score"], 4), "label": output["label"]} for output in outputs
]
print(outputs)

/home/seara/Desktop/Github/mtc-classification/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.5588, 'label': 'Hotel Room or living room or bedroom'}, {'score': 0.2495, 'label': 'Hotel Exterior or outdoor building'}, {'score': 0.0526, 'label': 'Conference room, Meeting room'}, {'score': 0.0451, 'label': 'Hotel Lobby or Reception area,'}, {'score': 0.0359, 'label': 'Corridors and stairs'}, {'score': 0.0259, 'label': 'Balcony'}, {'score': 0.0095, 'label': 'Pool or swimming pool'}, {'score': 0.0072, 'label': 'Bathroom or toilet'}, {'score': 0.0064, 'label': 'Restaurant'}, {'score': 0.0038, 'label': 'Spa and sauna or massage center or turkish bath'}, {'score': 0.0027, 'label': 'Beach or shore or coastline'}, {'score': 0.0012, 'label': 'Terrace and patio'}, {'score': 0.0011, 'label': 'Gym or Fitness center or exercise machine'}, {'score': 0.0003, 'label': 'Billiard or pool table'}, {'score': 0.0, 'label': 'Food, Food on a plate, Food on the table'}]


In [ ]:
folder_path = "public_test"

class_labels = [
    "Hotel Exterior or outdoor building",
    "Hotel Room or living room or bedroom",
    "Pool or swimming pool",
    "Billiard or pool table",
    "Bathroom or toilet",
    "Restaurant or the dining room at the hotel",
    "Hotel Lobby or reception area,",
    "Beach or shore or coastline",
    "Corridors and stairs",
    "Food or food on a plate or food on the table",
    "Conference room or a meeting room",
    "Gym or fitness center or exercise machine",
    "Balcony",
    "Terrace and patio, hotel courtyard",
    "Spa and sauna or massage center or turkish bath",
]

image_classifier = pipeline(
    task="zero-shot-image-classification",
    model="openai/clip-vit-large-patch14",
    device="cuda",
    batch_size=20,
)
# image_classifier(image_files, candidate_labels=class_labels)
image_files = [
    f"public_test/{f}"
    for f in os.listdir(folder_path)
    if f.endswith(("jpg", "jpeg", "png"))
]
outputs = image_classifier(image_files, candidate_labels=class_labels)

In [ ]:
threshold_for_zero_class = 0.1

new_outputs = [x[0] for x in outputs]
scores = [x["score"] for x in new_outputs]
labels = [x["label"] for x in new_outputs]
df = pd.DataFrame({"images": image_files, "labels": labels, "scores": scores})

df["class_number"] = df["labels"].apply(lambda x: class_labels.index(x)) + 1
df["class_number"] = df.apply(
    lambda x: x["class_number"] if x["scores"] > threshold_for_zero_class else 16,
    axis=1,
)
df["image_name"] = df["images"].apply(lambda x: x.split("/")[1])
df = df.sort_values(
    by="image_name", key=lambda x: [int(y.split("_")[2].split(".")[0]) for y in x]
)

df[["image_name", "class_number"]].to_csv(
    f"clip_openai_large_{threshold_for_zero_class}_new_prompts.csv", index=False
)
df

,images,labels,scores,class_number,image_name
199,public_test/public_test_1.jpg,Spa and sauna or massage center or turkish bath,0.914108,15,public_test_1.jpg
281,public_test/public_test_2.jpg,Hotel Room or living room or bedroom,0.761631,2,public_test_2.jpg
496,public_test/public_test_3.jpg,Hotel Room or living room or bedroom,0.978850,2,public_test_3.jpg
379,public_test/public_test_4.jpg,Gym or fitness center or exercise equipment,0.999753,12,public_test_4.jpg
458,public_test/public_test_5.jpg,"Terrace and patio, hotel courtyard",0.334291,14,public_test_5.jpg
...,...,...,...,...,...
334,public_test/public_test_1120.jpg,Conference room or a meeting room,0.998838,11,public_test_1120.jpg
326,public_test/public_test_1121.jpg,Gym or fitness center or exercise equipment,0.995384,12,public_test_1121.jpg
472,public_test/public_test_1122.jpg,Hotel Room or living room or bedroom,0.970078,2,public_test_1122.jpg
394,public_test/public_test_1123.jpg,Hotel Room or living room or bedroom,0.703739,2,public_test_1123.jpg


In [ ]:
df[["image_name", "labels", "scores"]].to_csv(
    f"clip_openai_large_{threshold_for_zero_class}.csv", index=False
)

In [ ]:
df

,images,labels,scores,classes
0,public_test/public_test_432.jpg,Hotel Room or living room or bedroom,0.805137,2
1,public_test/public_test_121.jpg,Terrace and patio,0.702430,14
2,public_test/public_test_930.jpg,"Hotel Lobby or reception area,",0.953016,7
3,public_test/public_test_1118.jpg,"Hotel Lobby or reception area,",0.661016,7
4,public_test/public_test_593.jpg,Terrace and patio,0.854167,14
...,...,...,...,...
1119,public_test/public_test_251.jpg,"Hotel Lobby or reception area,",0.996863,7
1120,public_test/public_test_711.jpg,Hotel Room or living room or bedroom,0.792319,2
1121,public_test/public_test_888.jpg,Terrace and patio,0.906344,14
1122,public_test/public_test_799.jpg,Hotel Room or living room or bedroom,0.955117,2


In [ ]:
temp = pd.read_csv("classification_results.csv")

In [ ]:
df

In [ ]:
new_outputs

[{'score': 0.8051366806030273,
  'label': 'Hotel Room or living room or bedroom'},
 {'score': 0.7024304270744324, 'label': 'Terrace and patio'},
 {'score': 0.9530163407325745, 'label': 'Hotel Lobby or reception area,'},
 {'score': 0.6610164642333984, 'label': 'Hotel Lobby or reception area,'},
 {'score': 0.8541666269302368, 'label': 'Terrace and patio'},
 {'score': 0.2758614420890808,
  'label': 'Restaurant or the dining room at the hotel'},
 {'score': 0.9838907122612, 'label': 'Hotel Lobby or reception area,'},
 {'score': 0.9163627028465271, 'label': 'Bathroom or toilet'},
 {'score': 0.8109623193740845, 'label': 'Corridors and stairs'},
 {'score': 0.8925807476043701,
  'label': 'Hotel Room or living room or bedroom'},
 {'score': 0.9900903701782227,
  'label': 'Restaurant or the dining room at the hotel'},
 {'score': 0.4027666449546814,
  'label': 'Hotel Room or living room or bedroom'},
 {'score': 0.9807963371276855, 'label': 'Beach or shore or coastline'},
 {'score': 0.42316347360610

In [ ]:
outputs[0]

[{'score': 0.8051366806030273,
  'label': 'Hotel Room or living room or bedroom'},
 {'score': 0.08630670607089996, 'label': 'Bathroom or toilet'},
 {'score': 0.044826261699199677, 'label': 'Corridors and stairs'},
 {'score': 0.04275964945554733, 'label': 'Balcony'},
 {'score': 0.009462676011025906, 'label': 'Terrace and patio'},
 {'score': 0.0035939221270382404, 'label': 'Hotel Lobby or reception area,'},
 {'score': 0.0027631819248199463,
  'label': 'Hotel Exterior or outdoor building'},
 {'score': 0.0024113820400089025,
  'label': 'Conference room or a meeting room'},
 {'score': 0.0010488524567335844,
  'label': 'Spa and sauna or massage center or turkish bath'},
 {'score': 0.0007427802192978561,
  'label': 'Gym or fitness center or exercise machine'},
 {'score': 0.0005330397398211062, 'label': 'Beach or shore or coastline'},
 {'score': 0.00019651296315714717, 'label': 'Pool or swimming pool'},
 {'score': 0.00010091483272844926,
  'label': 'Restaurant or the dining room at the hotel'}

In [ ]:
pd.DataFrame(outputs)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'score': 0.3609088659286499, 'label': 'Bedroom'}","{'score': 0.29851534962654114, 'label': 'Hallw...","{'score': 0.19883514940738678, 'label': 'Livin...","{'score': 0.07817911356687546, 'label': 'Bathr...","{'score': 0.021609248593449593, 'label': 'Balc...","{'score': 0.017162276431918144, 'label': 'Kitc...","{'score': 0.008442506194114685, 'label': 'Dini...","{'score': 0.004782090894877911, 'label': 'Terr...","{'score': 0.0031336841639131308, 'label': 'Rec...","{'score': 0.0027985882479697466, 'label': 'Bar'}","{'score': 0.002517841989174485, 'label': 'Offi...","{'score': 0.0011598514392971992, 'label': 'Spa'}","{'score': 0.0010143298422917724, 'label': 'Lob...","{'score': 0.000446272169938311, 'label': 'Gym'}","{'score': 0.0003972354461438954, 'label': 'Con...","{'score': 9.75503571680747e-05, 'label': 'Pool'}"
1,"{'score': 0.6982440948486328, 'label': 'Terrac...","{'score': 0.29048576951026917, 'label': 'Balco...","{'score': 0.002919814083725214, 'label': 'Dini...","{'score': 0.002343828324228525, 'label': 'Bedr...","{'score': 0.0013686706079170108, 'label': 'Bar'}","{'score': 0.001326257479377091, 'label': 'Rece...","{'score': 0.0012477835407480597, 'label': 'Kit...","{'score': 0.0007246978348121047, 'label': 'Liv...","{'score': 0.00048715845332480967, 'label': 'Of...","{'score': 0.0003130619879812002, 'label': 'Con...","{'score': 0.00018745272245723754, 'label': 'Po...","{'score': 0.00012644192611332983, 'label': 'Ha...","{'score': 8.75578771228902e-05, 'label': 'Bath...","{'score': 6.766915612388402e-05, 'label': 'Spa'}","{'score': 4.681444261223078e-05, 'label': 'Gym'}","{'score': 2.296349521202501e-05, 'label': 'Lob..."
2,"{'score': 0.6754939556121826, 'label': 'Lobby'}","{'score': 0.22994625568389893, 'label': 'Recep...","{'score': 0.018287504091858864, 'label': 'Hall...","{'score': 0.017293831333518028, 'label': 'Terr...","{'score': 0.0120464488863945, 'label': 'Bedroom'}","{'score': 0.011856157332658768, 'label': 'Conf...","{'score': 0.0096364701166749, 'label': 'Dining...","{'score': 0.008148983120918274, 'label': 'Bath...","{'score': 0.006400298327207565, 'label': 'Offi...","{'score': 0.004552432335913181, 'label': 'Livi...","{'score': 0.0022152469027787447, 'label': 'Bal...","{'score': 0.001979696797206998, 'label': 'Bar'}","{'score': 0.0016242478741332889, 'label': 'Gym'}","{'score': 0.0002401990641374141, 'label': 'Spa'}","{'score': 0.00018899580754805356, 'label': 'Ki...","{'score': 8.929752220865339e-05, 'label': 'Pool'}"
3,"{'score': 0.34113427996635437, 'label': 'Lobby'}","{'score': 0.21881619095802307, 'label': 'Hallw...","{'score': 0.16172575950622559, 'label': 'Recep...","{'score': 0.16024300456047058, 'label': 'Balco...","{'score': 0.05201197788119316, 'label': 'Terra...","{'score': 0.018842341378331184, 'label': 'Bar'}","{'score': 0.018328610807657242, 'label': 'Dini...","{'score': 0.012843920849263668, 'label': 'Livi...","{'score': 0.003588798688724637, 'label': 'Offi...","{'score': 0.003410617122426629, 'label': 'Bedr...","{'score': 0.002536688232794404, 'label': 'Conf...","{'score': 0.002496836706995964, 'label': 'Spa'}","{'score': 0.0024666700046509504, 'label': 'Gym'}","{'score': 0.0008458809461444616, 'label': 'Bat...","{'score': 0.00037926656659692526, 'label': 'Ki...","{'score': 0.0003292473847977817, 'label': 'Pool'}"
4,"{'score': 0.9048789739608765, 'label': 'Terrac...","{'score': 0.029473358765244484, 'label': 'Bar'}","{'score': 0.027461659163236618, 'label': 'Balc...","{'score': 0.018031243234872818, 'label': 'Rece...","{'score': 0.008208343759179115, 'label': 'Pool'}","{'score': 0.0033784392289817333, 'label': 'Con...","{'score': 0.0030391067266464233, 'label': 'Lob...","{'score': 0.0017494364874437451, 'label': 'Liv...","{'score': 0.0014328504912555218, 'label': 'Off...","{'score': 0.0011175228282809258, 'label': 'Din...","{'score': 0.0003866603074129671, 'label': 'Gym'}","{'score': 0.0003801360435318202, 'label': 'Kit...","{'score': 0.0001941861